In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

DATA_PATH = "dedup_data.csv"

df = pd.read_csv(DATA_PATH)
df.head()


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,id
0,mitchell,green,7.0,wallaby place,delmar,cleveland,2119,sa,19560409.0,1804974,74463
1,harley,mccarthy,177.0,pridhamstreet,milton,marsden,3165,nsw,19080419.0,6089216,60733
2,madeline,mason,54.0,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128.0,2185997,64831
3,isabelle,NaN,23.0,gundulu place,currin ga,utakarra,2193,wa,19921119.0,4314184,12416
4,taylor,hathaway,7.0,yuranigh court,brentwood vlge,NaN,4220,nsw,19991207.0,9144092,81570


In [2]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   given_name     4844 non-null   object 
 1   surname        4921 non-null   object 
 2   street_number  4755 non-null   float64
 3   address_1      4846 non-null   object 
 4   address_2      4307 non-null   object 
 5   suburb         4915 non-null   object 
 6   postcode       5000 non-null   int64  
 7   state          4915 non-null   object 
 8   date_of_birth  4845 non-null   float64
 9   soc_sec_id     5000 non-null   int64  
 10  id             5000 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 429.8+ KB


In [3]:
df.isna().sum()


given_name       156
surname           79
street_number    245
address_1        154
address_2        693
suburb            85
postcode           0
state             85
date_of_birth    155
soc_sec_id         0
id                 0
dtype: int64

In [4]:
%pip install rapidfuzz networkx --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from rapidfuzz import fuzz
import networkx as nx
import re

TEXT_COLUMNS = [
    "given_name",
    "surname",
    "address_1",
    "address_2",
    "suburb",
    "state",
]

NUMERIC_COLUMNS = [
    "street_number",
    "postcode",
]

IDENTIFIER_COLUMNS = ["id"]
META_COLUMNS = IDENTIFIER_COLUMNS

def normalize_text(value: str) -> str:
    if pd.isna(value):
        return ""
    value = value.strip().lower()
    value = re.sub(r"[^a-z0-9\s]", "", value)
    value = re.sub(r"\s+", " ", value)
    return value

df_clean = df.copy()
for col in TEXT_COLUMNS:
    df_clean[col] = df_clean[col].map(normalize_text)

for col in NUMERIC_COLUMNS:
    df_clean[col] = df_clean[col].fillna(0).astype(str).str.strip()

df_clean["date_of_birth"] = pd.to_datetime(df_clean["date_of_birth"], errors="coerce")
df_clean["dob_str"] = df_clean["date_of_birth"].dt.strftime("%Y-%m-%d").fillna("")
df_clean["soc_sec_id"] = df_clean["soc_sec_id"].fillna("").astype(str).str.strip()

df_clean.head()


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,id,dob_str
0,mitchell,green,7.0,wallaby place,delmar,cleveland,2119,sa,1970-01-01 00:00:00.019560409,1804974,74463,1970-01-01
1,harley,mccarthy,177.0,pridhamstreet,milton,marsden,3165,nsw,1970-01-01 00:00:00.019080419,6089216,60733,1970-01-01
2,madeline,mason,54.0,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,1970-01-01 00:00:00.019081128,2185997,64831,1970-01-01
3,isabelle,,23.0,gundulu place,currin ga,utakarra,2193,wa,1970-01-01 00:00:00.019921119,4314184,12416,1970-01-01
4,taylor,hathaway,7.0,yuranigh court,brentwood vlge,,4220,nsw,1970-01-01 00:00:00.019991207,9144092,81570,1970-01-01


In [6]:
from collections import defaultdict
from itertools import combinations


def build_blocks(data: pd.DataFrame, max_block_size: int = 150) -> dict:
    blocks: dict[str, list[int]] = defaultdict(list)
    for idx, row in data.iterrows():
        row_blocks = set()
        gid = row["id"]

        dob = row["dob_str"]
        given = row["given_name"]
        surname = row["surname"]
        postcode = row["postcode"]
        suburb = row["suburb"]
        state = row["state"]
        addr1 = row["address_1"]
        addr2 = row["address_2"]
        street = row["street_number"]
        ssn = row["soc_sec_id"]

        if ssn:
            row_blocks.add(f"ssn::{ssn}")

        if dob:
            row_blocks.add(f"dob::{dob}")
            if surname:
                row_blocks.add(f"dob_surname::{dob}::{surname[:4]}")
            if given:
                row_blocks.add(f"dob_given::{dob}::{given[:4]}")

        if postcode:
            row_blocks.add(f"postcode::{postcode}")
            if surname:
                row_blocks.add(f"surname_postcode::{surname[:5]}::{postcode}")

        if suburb:
            row_blocks.add(f"suburb::{suburb[:6]}")
            if street:
                row_blocks.add(f"suburb_street::{suburb[:6]}::{street}")

        if addr1:
            row_blocks.add(f"address1::{addr1[:8]}")

        if addr2:
            row_blocks.add(f"address2::{addr2[:6]}")

        if given and surname:
            row_blocks.add(f"name::{given[:4]}::{surname[:4]}")

        for key in row_blocks:
            blocks[key].append(gid)

    # prune oversized blocks
    return {k: v for k, v in blocks.items() if 1 < len(v) <= max_block_size}


blocks = build_blocks(df_clean)
len(blocks)


8510

In [7]:
def generate_candidate_pairs(blocks: dict[str, list[int]]) -> set[tuple[int, int]]:
    candidates: set[tuple[int, int]] = set()
    for records in blocks.values():
        if len(records) < 2:
            continue
        for a, b in combinations(sorted(records), 2):
            candidates.add((a, b))
    return candidates


candidate_pairs = generate_candidate_pairs(blocks)
len(candidate_pairs)


137592

In [8]:
record_lookup = df_clean.set_index("id")


def string_similarity(a: str, b: str) -> float:
    if not a and not b:
        return 1.0
    if not a or not b:
        return 0.0
    return fuzz.token_set_ratio(a, b) / 100.0


def short_string_similarity(a: str, b: str) -> float:
    if not a and not b:
        return 1.0
    if not a or not b:
        return 0.0
    return fuzz.partial_ratio(a, b) / 100.0


def exact_match_score(a: str, b: str) -> float:
    return float(a == b and a != "")


def dob_score(a: str, b: str) -> float:
    if not a and not b:
        return 0.0
    if not a or not b:
        return 0.0
    return float(a == b)


def compute_pair_features(id_a: int, id_b: int) -> dict:
    row_a = record_lookup.loc[id_a]
    row_b = record_lookup.loc[id_b]

    features = {
        "given_name": string_similarity(row_a["given_name"], row_b["given_name"]),
        "surname": string_similarity(row_a["surname"], row_b["surname"]),
        "address_1": string_similarity(row_a["address_1"], row_b["address_1"]),
        "address_2": string_similarity(row_a["address_2"], row_b["address_2"]),
        "suburb": string_similarity(row_a["suburb"], row_b["suburb"]),
        "state": short_string_similarity(row_a["state"], row_b["state"]),
        "street_number": exact_match_score(row_a["street_number"], row_b["street_number"]),
        "postcode": exact_match_score(row_a["postcode"], row_b["postcode"]),
        "dob": dob_score(row_a["dob_str"], row_b["dob_str"]),
        "soc_sec_id": exact_match_score(row_a["soc_sec_id"], row_b["soc_sec_id"]),
    }
    return features


def score_pair(features: dict) -> float:
    weights = {
        "given_name": 0.15,
        "surname": 0.2,
        "address_1": 0.15,
        "address_2": 0.05,
        "suburb": 0.1,
        "state": 0.05,
        "street_number": 0.05,
        "postcode": 0.1,
        "dob": 0.1,
        "soc_sec_id": 0.05,
    }
    total = 0.0
    weight_sum = 0.0
    for key, weight in weights.items():
        value = features[key]
        if np.isnan(value):
            continue
        total += weight * value
        weight_sum += weight
    return total / weight_sum if weight_sum else 0.0


# quick smoke-test on a sample of candidate pairs to inspect score distribution
sampled_pairs = list(candidate_pairs)[:10]
[score_pair(compute_pair_features(a, b)) for a, b in sampled_pairs]


[0.42547619047619045,
 0.39697792869269943,
 0.4153663003663004,
 0.9834545454545455,
 0.3890476190476191,
 0.35307017543859653,
 0.2870234604105572,
 0.5200641025641025,
 0.35285714285714287,
 0.44148351648351647]

In [11]:
# Ensure each id maps to a single record in lookups
record_lookup = df_clean.groupby("id", as_index=True).first()


In [12]:
def score_candidates(pairs: set[tuple[int, int]]) -> pd.DataFrame:
    rows = []
    for a, b in pairs:
        features = compute_pair_features(a, b)
        score = score_pair(features)
        row = {"id_a": a, "id_b": b, "score": score}
        row.update({f"feat_{k}": v for k, v in features.items()})
        rows.append(row)
    return pd.DataFrame(rows)


pair_scores = score_candidates(candidate_pairs)
pair_scores.describe().T


,count,mean,std,min,25%,50%,75%,max
id_a,137592.0,33091.999346,23465.312577,36.000000,13266.000000,28632.000000,50337.000000,99776.0
id_b,137592.0,66319.434655,23432.294614,241.000000,50210.000000,70121.000000,85833.000000,99992.0
score,137592.0,0.407983,0.115112,0.064474,0.348989,0.395385,0.439803,1.0
feat_given_name,137592.0,0.506674,0.361800,0.000000,0.200000,0.363636,1.000000,1.0
feat_surname,137592.0,0.455659,0.363454,0.000000,0.166667,0.307692,0.933333,1.0
feat_address_1,137592.0,0.403541,0.230783,0.000000,0.266667,0.344828,0.518519,1.0
feat_address_2,137592.0,0.279743,0.265928,0.000000,0.111111,0.242424,0.342857,1.0
feat_suburb,137592.0,0.352391,0.270559,0.000000,0.190476,0.272727,0.380952,1.0
feat_state,137592.0,0.348571,0.425620,0.000000,0.000000,0.000000,0.666667,1.0
feat_street_number,137592.0,0.047830,0.213407,0.000000,0.000000,0.000000,0.000000,1.0


In [13]:
pair_scores["score"].quantile([0.5, 0.75, 0.9, 0.95, 0.98, 0.99])


0.50    0.395385
0.75    0.439803
0.90    0.485317
0.95    0.539738
0.98    0.870863
0.99    0.938095
Name: score, dtype: float64

In [14]:
pair_scores.sort_values("score", ascending=False).head(10)


,id_a,id_b,score,feat_given_name,feat_surname,feat_address_1,feat_address_2,feat_suburb,feat_state,feat_street_number,feat_postcode,feat_dob,feat_soc_sec_id
75711,33467,55217,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
37992,69310,79644,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75836,2457,53434,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
86978,49770,95087,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
84249,16095,79036,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
20617,4205,80175,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
91206,8923,77923,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
42240,27454,46281,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
19927,45151,76582,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
69556,88757,96139,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [15]:
THRESHOLD = 0.82

matching_pairs = pair_scores[pair_scores["score"] >= THRESHOLD][["id_a", "id_b", "score"]]
matching_pairs.head()


,id_a,id_b,score
3,7646,98972,0.983455
22,15659,94798,0.913856
229,57524,83035,0.894000
243,53366,54073,0.883766
392,55118,55595,0.988636


In [16]:
len(matching_pairs)


3749

In [17]:
graph = nx.Graph()

graph.add_nodes_from(df_clean["id"].tolist())
graph.add_weighted_edges_from(matching_pairs.itertuples(index=False, name=None))

components = list(nx.connected_components(graph))
len(components)


2607

In [18]:
cluster_assignments = []
for cluster_idx, component in enumerate(components, start=1):
    for record_id in component:
        cluster_assignments.append({"id": record_id, "cluster_id": cluster_idx})

clusters_df = pd.DataFrame(cluster_assignments)
clusters_df.head()


,id,cluster_id
0,74463,1
1,56487,2
2,52361,2
3,39470,2
4,7605,2


In [19]:
df_with_clusters = df.merge(clusters_df, on="id", how="left")
df_with_clusters["cluster_id"] = df_with_clusters["cluster_id"].fillna(-1).astype(int)
df_with_clusters.head()


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,id,cluster_id
0,mitchell,green,7.0,wallaby place,delmar,cleveland,2119,sa,19560409.0,1804974,74463,1
1,harley,mccarthy,177.0,pridhamstreet,milton,marsden,3165,nsw,19080419.0,6089216,60733,2
2,madeline,mason,54.0,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128.0,2185997,64831,3
3,isabelle,NaN,23.0,gundulu place,currin ga,utakarra,2193,wa,19921119.0,4314184,12416,4
4,taylor,hathaway,7.0,yuranigh court,brentwood vlge,NaN,4220,nsw,19991207.0,9144092,81570,5


In [20]:
cluster_sizes = df_with_clusters.groupby("cluster_id").size().reset_index(name="count").sort_values("count", ascending=False)
cluster_sizes.head(10)


,cluster_id,count
893,894,8
34,35,7
883,884,7
433,434,7
923,924,7
1359,1360,7
1700,1701,7
698,699,6
782,783,6
599,600,6


In [21]:
multi_record_clusters = cluster_sizes[cluster_sizes["count"] > 1]
len(multi_record_clusters), multi_record_clusters.head()


(1092,
      cluster_id  count
 893         894      8
 34           35      7
 883         884      7
 433         434      7
 923         924      7)

In [22]:
example_cluster_id = multi_record_clusters.iloc[0]["cluster_id"]
df_with_clusters[df_with_clusters["cluster_id"] == example_cluster_id].sort_values("id")


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,id,cluster_id
2563,emma,golder,381.0,fullerton crescent,carey park,campsie,2107,qld,19260419.0,1677968,6844,894
1813,emma,golfer,4.0,fullerton crescent,carey park,campyie,2107,qld,19260419.0,1677968,24802,894
1819,braiden,goldsworpthy,12.0,matina street,NaN,tumbarumba,2161,nsw,19501119.0,7570745,24802,894
3589,emma,golder,381.0,fullerton crescent,carey oark,campsie,2107,qld,19260419.0,1677968,28266,894
3539,emma,goldet,381.0,fullerton crescent,NaN,campsie,2107,qld,19260419.0,1677968,51437,894
1094,emma,golder,381.0,fullerton crescent,careybpark,campsie,2107,qld,19260419.0,1677968,56243,894
2652,emma,goldesr,381.0,fullerton crescent,care ypark,campsie,2107,qld,19260419.0,1677968,68901,894
4173,jenna,pasailc,679.0,tarana stnreet,mallanganee,osbofne,4342,vic,19541124.0,7730932,68901,894


In [23]:
OUTPUT_PATH = "dedup_clusters.csv"
df_with_clusters.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH


'dedup_clusters.csv'

## Notes on Deduplication Strategy

- **Preprocessing**: Normalized text fields (casefolding, punctuation stripping) and standardized numeric/date identifiers.
- **Blocking**: Generated multiple blocking keys (DOB, postcode, name fragments, SSN, suburb) to limit candidate pairs while capturing potential duplicates.
- **Scoring**: Used RapidFuzz similarity metrics and rule-based exact matches to compute a weighted similarity score per candidate pair.
- **Clustering**: Treated high-scoring pairs as edges in a graph and used connected components to assign cluster IDs representing deduplicated persons.
- **Output**: Saved enriched dataset with `cluster_id` assignments to `dedup_clusters.csv` for downstream use.
